In [4]:
pip install fastapi uvicorn

In [5]:
!pip install pyngrok

/bin/bash: line 1: pip install pyngrok: command not found


In [2]:
!nvidia-smi
!pip install --upgrade torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118


/bin/bash: line 1: nvidia-smi: command not found
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 838.4/838.4 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 7.8 

In [1]:
!nvidia-smi


/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
from IPython import get_ipython
from IPython.display import display
# %%
%cd /content
!git clone -b totoro4 https://github.com/camenduru/ComfyUI /content/TotoroUI
%cd /content/TotoroUI

!pip install -q torchsde einops diffusers accelerate xformers==0.0.28.post2 gradio==3.50.2 python-multipart==0.0.12 pyngrok
!apt -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev-fp8-all-in-one.safetensors -d /content/TotoroUI/models/checkpoints -o flux1-dev-fp8-all-in-one.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux_realism_lora.safetensors -d /content/TotoroUI/models/loras -o flux_realism_lora.safetensors

import random
import torch
import numpy as np
from PIL import Image
import nodes
from nodes import NODE_CLASS_MAPPINGS
from totoro_extras import nodes_custom_sampler
from totoro_extras import nodes_flux
from totoro import model_management
import gradio as gr

CheckpointLoaderSimple = NODE_CLASS_MAPPINGS["CheckpointLoaderSimple"]()
LoraLoader = NODE_CLASS_MAPPINGS["LoraLoader"]()
FluxGuidance = nodes_flux.NODE_CLASS_MAPPINGS["FluxGuidance"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()

with torch.inference_mode():
    unet, clip, vae = CheckpointLoaderSimple.load_checkpoint("flux1-dev-fp8-all-in-one.safetensors")

def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2

@torch.inference_mode()
def generate(positive_prompt, width, height, seed, steps, sampler_name, scheduler, guidance, lora_strength_model, lora_strength_clip):
    global unet, clip
    if seed == 0:
        seed = random.randint(0, 18446744073709551615)
    print(seed)
    unet_lora, clip_lora = LoraLoader.load_lora(unet, clip, "flux_realism_lora.safetensors", lora_strength_model, lora_strength_clip)
    cond, pooled = clip_lora.encode_from_tokens(clip_lora.tokenize(positive_prompt), return_pooled=True)
    cond = [[cond, {"pooled_output": pooled}]]
    cond = FluxGuidance.append(cond, guidance)[0]
    noise = RandomNoise.get_noise(seed)[0]
    guider = BasicGuider.get_guider(unet_lora, cond)[0]
    sampler = KSamplerSelect.get_sampler(sampler_name)[0]
    sigmas = BasicScheduler.get_sigmas(unet_lora, scheduler, steps, 1.0)[0]
    latent_image = EmptyLatentImage.generate(closestNumber(width, 16), closestNumber(height, 16))[0]
    sample, sample_denoised = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)
    decoded = VAEDecode.decode(vae, sample)[0].detach()
    Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save("/content/flux.png")
    return "/content/flux.png"

# with gr.Blocks(analytics_enabled=False) as demo:
#     with gr.Row():
#         with gr.Column():
#             positive_prompt = gr.Textbox(lines=3, interactive=True, value="cute anime girl ", label="Prompt")
#             width = gr.Slider(minimum=256, maximum=2048, value=1024, step=16, label="width")
#             height = gr.Slider(minimum=256, maximum=2048, value=1024, step=16, label="height")
#             seed = gr.Slider(minimum=0, maximum=18446744073709551615, value=0, step=1, label="seed (0=random)")
#             steps = gr.Slider(minimum=4, maximum=50, value=20, step=1, label="steps")
#             guidance = gr.Slider(minimum=0, maximum=20, value=3.5, step=0.5, label="guidance")
#             lora_strength_model = gr.Slider(minimum=0, maximum=1, value=1.0, step=0.1, label="lora_strength_model")
#             lora_strength_clip = gr.Slider(minimum=0, maximum=1, value=1.0, step=0.1, label="lora_strength_clip")
#             sampler_name = gr.Dropdown(["euler", "heun", "heunpp2", "heunpp2", "dpm_2", "lms", "dpmpp_2m", "ipndm", "deis", "ddim", "uni_pc", "uni_pc_bh2"], label="sampler_name", value="euler")
#             scheduler = gr.Dropdown(["normal", "sgm_uniform", "simple", "ddim_uniform"], label="scheduler", value="simple")
#             generate_button = gr.Button("Generate")
#         with gr.Column():
#             output_image = gr.Image(label="Generated image", interactive=False)

#     generate_button.click(fn=generate, inputs=[positive_prompt, width, height, seed, steps, sampler_name, scheduler, guidance, lora_strength_model, lora_strength_clip], outputs=output_image)

# demo.queue().launch(inline=False, share=True, debug=True)
# %%
# pip install fastapi uvicorn pyngrok
# %%
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import FileResponse
import uvicorn
from pyngrok import ngrok

# ... (Your existing ComfyUI code here)

app = FastAPI()

# ... (Your existing FastAPI routes here)
@app.post("/generate")
async def generate_image(
    positive_prompt: str,
    width: int,
    height: int,
    seed: int = 0,
    steps: int = 20,
    sampler_name: str = "euler",
    scheduler: str = "simple",
    guidance: float = 3.5,
    lora_strength_model: float = 1.0,
    lora_strength_clip: float = 1.0,
):
    image_path = generate(positive_prompt, width, height, seed, steps, sampler_name, scheduler, guidance, lora_strength_model, lora_strength_clip)
    return FileResponse(image_path, media_type="image/png")
if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    # Set your ngrok auth token
    ngrok.set_auth_token("2d0DqdzmBcQdT3ZX94qpUMtukI7_tkQwvfo4984kt3GnMZmT")  # Replace with your actual auth token
    # Open a ngrok tunnel to the FastAPI server
    ngrok_tunnel = ngrok.connect(8000)
    print('Public URL:', ngrok_tunnel.public_url)
    # Change "test:app" to "__main__:app"
    uvicorn.run("__main__:app", host="0.0.0.0", port=8000) # This line was changed


/content
fatal: destination path '/content/TotoroUI' already exists and is not an empty directory.
/content/TotoroUI
aria2 is already the newest version (1.36.0-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
c3d4dc|OK  |       0B/s|/content/TotoroUI/models/checkpoints/flux1-dev-fp8-all-in-one.safetensors

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
902869|OK  |       0B/s|/content/TotoroUI/models/loras/flux_realism_lora.safetensors

Status Legend:
(OK):download completed.


INFO:     Started server process [5703]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: https://c73a-34-105-89-12.ngrok-free.app
INFO:     2405:201:8011:e0d1:b000:156f:d2b2:fa3e:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2405:201:8011:e0d1:b000:156f:d2b2:fa3e:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2405:201:8011:e0d1:b000:156f:d2b2:fa3e:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2405:201:8011:e0d1:b000:156f:d2b2:fa3e:0 - "GET /openapi.json HTTP/1.1" 200 OK
13852344091764009548


  0%|          | 0/20 [00:00<?, ?it/s]

INFO:     2405:201:8011:e0d1:b000:156f:d2b2:fa3e:0 - "POST /generate?positive_prompt=A%20serene%20forest%20clearing%20during%20golden%20hour%2C%20with%20rays%20of%20sunlight%20streaming%20through%20tall%2C%20ancient%20trees.%20A%20sparkling%20brook%20runs%20through%20the%20scene%2C%20surrounded%20by%20lush%20green%20moss%20and%20colorful%20wildflowers.%20A%20mystical%20aura%20is%20created%20by%20soft%20glowing%20orbs%20floating%20above%20the%20water%2C%20while%20a%20small%20wooden%20bridge%20arcs%20gracefully%20over%20the%20stream.%20The%20style%20is%20highly%20detailed%20and%20photorealistic%2C%20evoking%20a%20magical%2C%20tranquil%20atmosphere.&width=1024&height=1024&seed=0&steps=20&sampler_name=euler&scheduler=simple&guidance=3.5&lora_strength_model=1&lora_strength_clip=1 HTTP/1.1" 200 OK
18149492124852491087


  0%|          | 0/20 [00:00<?, ?it/s]

INFO:     2405:201:8011:e0d1:b000:156f:d2b2:fa3e:0 - "POST /generate?positive_prompt=Magical%20forest&width=1024&height=1024&seed=0&steps=20&sampler_name=euler&scheduler=simple&guidance=3.5&lora_strength_model=1&lora_strength_clip=1 HTTP/1.1" 200 OK
15054957829543698201


  0%|          | 0/20 [00:00<?, ?it/s]

INFO:     2405:201:8011:e0d1:b000:156f:d2b2:fa3e:0 - "POST /generate?positive_prompt=wooden%20bridge&width=1024&height=1024&seed=0&steps=20&sampler_name=euler&scheduler=simple&guidance=3.5&lora_strength_model=1&lora_strength_clip=1 HTTP/1.1" 200 OK
